<a href="https://colab.research.google.com/github/here0009/An-Introduction-To-Applied-Bioinformatics/blob/master/hertie_transformers_gpt3_perplexity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Measuring change in perplexity with GPT-3

<br><br>

## Import (and install) necessary libraries

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install torch==1.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import argparse
from collections import defaultdict
from datetime import datetime
import gdown
import gzip
import json
import os
import pickle
import random
import re
import shutil

import pandas as pd

In [ ]:
import torch
from tqdm import tqdm

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from transformers import TextDataset, LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, AutoTokenizer

<br><br>

## Function to get perplexity of a sequence

This is from https://huggingface.co/docs/transformers/perplexity

This is a bit complicated because we need to account for the fixed length of GPT-2. To measure the perplexity of the full test sequence (which might be longer than 512 tokens), we need to use a sliding window.

In [ ]:
def get_perplexity(model, device, encodings):
  
  max_length = model.config.n_positions
  stride = 512

  nlls = []
  for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
      begin_loc = max(i + stride - max_length, 0)
      end_loc = min(i + stride, encodings.input_ids.size(1))
      trg_len = end_loc - i  # may be different from stride on last loop
      input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
      target_ids = input_ids.clone()
      target_ids[:, :-trg_len] = -100

      with torch.no_grad():
          outputs = model(input_ids, labels=target_ids)
          neg_log_likelihood = outputs[0] * trg_len

      nlls.append(neg_log_likelihood)

  ppl = torch.exp(torch.stack(nlls).sum() / end_loc)

  return ppl

<br><br>

## Function to fine-tune a model

This function loads a pretrained GPT-2 model, loads a dataset, and fine-tunes the model on the dataset. Finally, we save the model for later use.

The parameters are left at their default values, but you should experiment with changing them.

In [ ]:
def fine_tune(target_tag, model_id, device):

    model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    test_dataset = TextDataset(tokenizer=tokenizer,
                                file_path=target_tag + '.txt',
                                block_size=128)

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    training_args = TrainingArguments(output_dir="/sample_data", #The output directory
                                        overwrite_output_dir=True, #overwrite the content of the output directory
                                        num_train_epochs=3, # number of training epochs
                                        per_device_train_batch_size=16, # batch size for training
                                        per_device_eval_batch_size=32,  # batch size for evaluation
                                        eval_steps=400, # Number of update steps between two evaluations.
                                        save_steps=800, # after # steps model is saved
                                        warmup_steps=10 # number of warmup steps for learning rate scheduler
                                    )

    trainer = Trainer(model=model,
                        args=training_args,
                        data_collator=data_collator,
                        train_dataset=test_dataset,
                        eval_dataset=test_dataset)

    trainer.train()
    trainer.save_model('gpt2-' + '-'.join(target_tag.split()))

    del model

<br><br>

## Function to get fine-tuned perplexities

Here we extract the perplexity score for each review from the fine-tuned model. We save these scores in an ordered list.

In [ ]:
def get_finetuned_perplexities(target_tag, device, model_id, sentences):

    finetuned_perplexities = []

    finetuned_model = GPT2LMHeadModel.from_pretrained('gpt2-' + '-'.join(target_tag.split())).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    for _sentence in sentences:
        _encoding = tokenizer(_sentence, return_tensors='pt', return_length=True, truncation=True)
        _finetuned_perplexity = get_perplexity(finetuned_model, device, _encoding).item()
        finetuned_perplexities.append(_finetuned_perplexity)

    return finetuned_perplexities

<br><br>

## Load a test dataset

We load the same UCSD book review dataset that we used in our classification example. This data includes Goodreads book reviews across different genres. Using the full dataset, we can get many more genres than these.

In [ ]:
# This is where our target data is hosted on the web. You only need these paths for the book review dataset.

genre_url_dict = {'poetry':                 'https://drive.google.com/uc?id=1FVD3LxJXRc5GrKm97LehLgVGbRfF9TyO',
                  'children':               'https://drive.google.com/uc?id=1908GDMdrhDN7sTaI_FelSHxbwcNM1EzR',
                  'comics_graphic':         'https://drive.google.com/uc?id=1V4MLeoEiPQdocCbUHjR_7L9ZmxTufPFe',
                  'fantasy_paranormal':     'https://drive.google.com/uc?id=1THnnmE4XSCvMkGOsqapQr2cJI5amKA6X',
                  'history_biography':      'https://drive.google.com/uc?id=1lDkTzM6zpYU-HGkVAQgsw0dBzik-Zde9',
                  'mystery_thriller_crime': 'https://drive.google.com/uc?id=1ONpyuv0vrtd6iUEp0-zzkKqwpm3njEqi',
                  'romance':                'https://drive.google.com/uc?id=1NpFsDQKBj_lrTzSASfyKbmkSykzN88wE',
                  'young_adult':            'https://drive.google.com/uc?id=1M5iqCZ8a7rZRtsmY5KQ5rYnP9S0bQJVo'}

In this example, we'll focus just on the poetry reviews.

In [ ]:
gdown.download(genre_url_dict['poetry'], 'poetry.json.gz', quiet=False) 

Downloading...
From: https://drive.google.com/uc?id=1FVD3LxJXRc5GrKm97LehLgVGbRfF9TyO
To: /content/poetry.json.gz
100%|██████████| 49.3M/49.3M [00:00<00:00, 302MB/s]


'poetry.json.gz'

In [ ]:
def load_reviews(file_name,
                 head=None):

    reviews = []
    count = 0

    with gzip.open(file_name) as file:
        for line in file:

            d = json.loads(line)
            count += 1

            _book_id = d['book_id']

            reviews.append(d['review_text'])

            # Break if we reach the Nth line
            if (head is not None) and (count > head):
                break

    return reviews

In [ ]:
reviews = load_reviews('poetry.json.gz')
reviews = random.sample(reviews, 200)

Let's take a look at a random review.

In [ ]:
print(random.sample(reviews, 1)[0])

Great writing by MS Woodson. Enjoyed this book about growing up from the perspective of a young girl.


In [ ]:
flattened_reviews = ' '.join(reviews)

<br><br>

## Load the gpt-2 model and get the original perplexities of the reviews

In [ ]:
device = 'cuda'
model_id = 'gpt2'

model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
original_perplexities = []
for i, _review in enumerate(reviews):
    _encoding = tokenizer(_review, return_tensors='pt', return_length=True, truncation=True)
    original_perplexities.append(get_perplexity(model, device, _encoding).item())

100%|██████████| 1/1 [00:00<00:00, 68.33it/s]


<br><br>

## Fine-tune the model on the reviews

We fine-tune our pretrained model on a set of poetry reviews. We can watch the fine-tuning process and the training loss in the notebook.

In [ ]:
# Create a file containing the training data
dataset_file = open('poetry.txt', 'w')
dataset_file.write(flattened_reviews)
dataset_file.close()

# Fine-tune on the file created above
fine_tune('poetry', model_id, device)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (22718 > 1024). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 177
  Num Epochs = 3
  Instantaneous batch size per device = 16
  To

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to gpt2-poetry
Configuration saved in gpt2-poetry/config.json
Model weights saved in gpt2-poetry/pytorch_model.bin


<br><br>

## Get the new perplexities of the reviews

Just as we did for the original perplexities, we'll measure the perplexity of each review after fine-tuning the pretrained model.

In [ ]:
finetuned_perplexities = get_finetuned_perplexities('poetry', device, model_id, reviews)

loading configuration file gpt2-poetry/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.21.1",
  "use_cache": true,
  "vocab_size": 50257
}

loading weights f

Let's re-arrange all our perplexity data into a single pandas dataframe. This will allow us to examine the data (and differences between the original and fine-tuned perplexities) more easily.

In [ ]:
perplexity_output_dicts = []
for i, p in enumerate(finetuned_perplexities):
    perplexity_output_dicts.append({'fine_tuned_tag': 'poetry',
                                    'review': reviews[i],
                                    'original_ppl': original_perplexities[i],
                                    'new_ppl': p})
perplexity_output_df = pd.DataFrame(perplexity_output_dicts)
perplexity_output_df.to_csv('perplexity_output.poetry.csv')

In [ ]:
perplexity_output_df.sample(3)

,fine_tuned_tag,review,original_ppl,new_ppl
160,poetry,"Well, that was frickin' brilliant.",23.303785,16.099445
14,poetry,I had this title in mind for a while after rea...,29.351795,20.276806
109,poetry,So amazing! \n This book hit me deeply on so m...,75.824944,33.489891


In [ ]:
perplexity_output_df.describe()

,original_ppl,new_ppl
count,200.000000,200.000000
mean,284.429726,95.723942
std,987.660772,147.647705
min,11.465781,9.088837
25%,41.840387,28.522824
50%,77.779865,50.149952
75%,225.305412,103.709164
max,12867.959961,1312.637573


In [ ]:
perplexity_output_df['change_ppl'] = perplexity_output_df['new_ppl'] - perplexity_output_df['original_ppl']

Which reviews' perplexity scores have improved the most after fine-tuning? Remember that we fine-tuned and tested on the same set of reviews, so we expect everything to improve.

Let's print the review texts and think about why these reviews improve the most.

In [ ]:
perplexity_output_df = perplexity_output_df.sort_values(by=['change_ppl'], ascending=True)

for i, r in perplexity_output_df.head(10).iterrows():
    print(round(r['change_ppl'], 2), '\t', ' '.join(r['review'].split()))
print('...')
for i, r in perplexity_output_df.tail(10).iterrows():
    print(round(r['change_ppl'], 2), '\t', ' '.join(r['review'].split()))

-12578.75 	 swnt lGyb ! !
-2661.87 	 keingat guru Bahasa Indonesia sewaktu SMA... inget semangat ngajarnya...
-2084.62 	 lbr@ f~ Hb llh
-1712.72 	 m 'nt swy fs `dyh lknh l tkl bsr`@ wDrbth `nydh
-1459.25 	 mmlun ,, wkf~ !!
-917.39 	 fltrHm~ D`f~ ... wql@ Hylt~ hdh `tb lHb ...... ll'Hbb
-844.77 	 w nWy 'uHbk nt bdy@ rwHy wnt lkhtm , yTyr lHmm yHT lHmm ..
-690.49 	 my favourite poem , ever !!!
-648.16 	 Hy'run 'n byn 'n ybd' lfrH w'laW ybd' mkhf@a ynthy .
-598.96 	 mn 'jml m qr't =)
...
-5.26 	 I love the darkness of Poe's short stories. If you haven't read them, you are simply missing out.
-4.6 	 My favorite of everything I've read of his so far. Which isn't that much, but I'm in a much better mood now.
-4.53 	 As a complete work I would give part I and part II 3 stars. If I could rate the parts separately I would give the first part 4 stars and the second part 2 stars. I liked the first part a lot - a lot of things were happening but the story was still consistent. I did not like the s

<br><br>

## Examine perplexities when fine-tuning on different genres

If we re-ran the above code with many different genres, where we fine-tuned and tested the perplexity on each pair of genres, we might get output like the following.

Each cell entry in this heatmap represents a change in perplexity. The genres on the y-axis represent the genre used for fine-tuning, while the genres on the x-axis represent the genre's whose perlexity values were tested.

In [ ]:
from IPython.display import Image
Image('heatmap.jpg')